# Getting images using the new SIAv2 application

In [ ]:
import os
import random
import requests
from pyvo.dal import SIA2Service
import matplotlib.pyplot as plt

import pyvo
from astropy.io import fits
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import imshow_norm, ZScaleInterval
from astropy.visualization.stretch import  SqrtStretch
from lsst.rsp import get_datalink_result
from lsst.rsp.utils import get_pyvo_auth, get_access_token, get_service_url

In [ ]:
# Specify the SIA service to use
sia_url = "https://data-int.lsst.cloud/api/sia/dp02"

In [ ]:
# Setup authentication
s = requests.Session()
tok = get_access_token()
s.headers["Authorization"] = f"Bearer {tok}"
auth = pyvo.auth.authsession.AuthSession()
auth.credentials.set("lsst-token", s)
auth.add_security_method_for_url(sia_url + "/query", "lsst-token")
auth.add_security_method_for_url(get_service_url("datalink"), "lsst-token")

In [ ]:
# Get an object to query the SIAV2 service with
service = SIA2Service(sia_url, auth)

In [ ]:
# Execute a query looking for pictures within a circle
# and show them as a table

t1 = Time("60550.31803461111", format='mjd').to_datetime()
t2 = Time("60550.31838182871", format='mjd').to_datetime()

results = service.search(pos=(55.7467, -32.2862, 0.05), time=[t1, t2])
results.to_table()

In [ ]:
# Extract the access URL from the result in the first row
dataLinkUrl = random.choice(results).access_url

In [ ]:
from pyvo.dal.adhoc import DatalinkResults
dl_result = DatalinkResults.from_result_url(
    dataLinkUrl, session=auth
)

In [ ]:
# Retrieve the datalink votable document and
# show it in the notebook
dl_result

In [ ]:
# Full image of calexp - not a cutout
image_url = dl_result.getrecord(0).get('access_url')
image_url

In [ ]:
# Now let's download the image 
filename = download_file(image_url)
filename

In [ ]:
hdulist = fits.open(image_url)

for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])